In [ ]:
import time
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from concurrent import futures

In [ ]:
data=[]
options = Options()
options.add_argument('--headless=new')

def get_comments(url, options=None):
    with Chrome(options=options) as driver:
        wait = WebDriverWait(driver,5)
        driver.get(url)

        for item in range(15): 
            wait.until(EC.visibility_of_element_located((By.TAG_NAME, "body"))).send_keys(Keys.END)
            time.sleep(1)

        for comment in wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#content-text"))):
            data.append(comment.text)
    return data

urls = [
    'https://www.youtube.com/watch?v=wSTYIyQxfPQ',
    'https://www.youtube.com/watch?v=TNsSBhl_2LI',
    'https://www.youtube.com/watch?v=DviID8Ni7Ns',
    'https://www.youtube.com/watch?v=9IZKcb3LndA',
]

comments = []
with futures.ThreadPoolExecutor() as executor:
    future_comments = [executor.submit(get_comments, url, options) for url in urls]
    for future_comment, link in zip(future_comments, urls): 
        try:        
            comments += future_comment.result() # can use `timeout` to wait max seconds for each thread               
        except Exception as exc:
            print('url {:0} generated an exception: {:1}'.format(link, exc))

In [ ]:
comments

In [ ]:
with open("datasets/comments.txt", "w", encoding='utf8') as output:
    output.write('\n'.join([comment.replace('\n', ' ') for comment in data]))

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp(' '.join(data))
adjectives = [token.text for token in doc if token.pos_ == 'ADJ']

In [ ]:
from collections import Counter
Counter(adjectives)